# Final_Model_2a_Sbert_Transformer_Cosine_MediaSum_Extractive_Summarization
    Oct 31, 2022




This notebook has the following model built for an extractive summarizer based on Bert Sentence Transfomer Model from Hugging Face.

1. SBertSummarizer works by first embedding the sentences, then running a clustering algorithm, finding the sentences that are closest to the cluster's centroids. This library also uses coreference techniques, utilizing the https://github.com/huggingface/neuralcoref library to resolve words in summaries that need more context. 
2.SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. This framework can be used to compute sentence / text embeddings for more than 100 languages. **These** embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.

Dataset Summary

This large-scale media interview dataset contains 463.6K transcripts with abstractive summaries, collected from interview transcripts and overview / topic descriptions from NPR and CNN.

Data Fields

    id: paper id
    document: a string/list containing the body of a set of documents
    summary: a string containing the abstract of the set

Model Details

    Sentence Transformer: Sentence Transformer
    Pre-Training: all-MiniLM-L6-v2
    Classification: Degree Centrality of Sentences in a Document
    Trigram Blocking: Yes
    Fine Tuning: None
    Evaluation Metrics: RougeL and Cosine Similarity
    

# 1. Setup

#### This section install key libraries

In [ ]:
!pip install bert-extractive-summarizer --quiet

     |████████████████████████████████| 5.3 MB 19.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 
     |████████████████████████████████| 163 kB 87.6 MB/s 


In [ ]:
!pip install datasets --quiet
!pip install nltk --quiet

     |████████████████████████████████| 441 kB 14.6 MB/s 
     |████████████████████████████████| 115 kB 76.7 MB/s 
     |████████████████████████████████| 212 kB 101.2 MB/s 
     |████████████████████████████████| 95 kB 5.1 MB/s 
     |████████████████████████████████| 127 kB 70.5 MB/s 
     |████████████████████████████████| 115 kB 84.3 MB/s 


In [ ]:
!pip install -q rouge_score

In [ ]:
!pip install -q evaluate


     |████████████████████████████████| 72 kB 840 kB/s 


In [ ]:
!pip install -U spacy
!pip install -U spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98.5 MB 191 kB/s 


In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score
!pip install rouge_score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_lg

2022-11-01 03:06:50.823444: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# 2.0 Import Libraries

In [ ]:
# NLTK
import re # relugar expression
import nltk # natural language toolkit for sentence tokenization and display
import string
import heapq
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.util import ngrams
import evaluate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os
import logging

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
import evaluate

In [ ]:
import spacy
import pandas as pd

print(spacy.__version__)
print(pd.__version__)

3.4.2
1.3.5


In [ ]:
import pickle
import subprocess
import sys
import nltk
from nltk import Nonterminal, nonterminals, Production, CFG, PCFG

In [ ]:
#shift reduce parser example
from nltk.grammar import Nonterminal
from nltk.parse.api import ParserI
from nltk.tree import Tree

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
import time

In [ ]:
# Mount drive for saving model checkpoints, loading Task 2 data below

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3.0 Load the dataset



In [ ]:
from datasets import load_dataset, load_metric


In [ ]:
dataset_id = "ccdv/mediasum"
dataset = load_dataset(dataset_id, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset mediasum downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___mediasum/roberta_prepended/1.0.0/46142d4b8658fecd0c985e9f5f070b3e37b42ed7feabd04682dce334bd5e526c. Subsequent calls will reuse this data.


In [ ]:
# Inspect data structure
print(dataset)

Dataset({
    features: ['document', 'summary'],
    num_rows: 443596
})


In [ ]:
# Inspect Shape
print(dataset.shape)



(443596, 2)


# 4.0 Inspect MediaSum

In [ ]:
# inspect first example
dataset[0]

{'document': 'FARAI CHIDEYA, host: Now, moving on, Forest Whitaker as Moses, Tisha Campbell Martin as Mary Magdalene - well, that\'s all in "The Bible Experience." A New Testament edition was released in 2006. This edition is billed as "The Complete Bible." It doesn\'t have one person reading the gospels. It features nearly 400 African-American artists, actors and ministers, plus sound effects.</s>FARAI CHIDEYA, host: Just listen to Blair Underwood\'s rendition of Jesus on the cross.</s>Mr. BLAIR UNDERWOOD (Actor): (As Jesus) My God, my God, why have you forsaken me?</s>FARAI CHIDEYA, host: Now, we\'ve got two people affiliated with the project with us today. Kyle Bowser, he co-produced "The Bible Experience" and actress Wendy Raquel Robinson, one of the actors in "The Bible Experience," and she also stars in the CW series, "The Game."</s>FARAI CHIDEYA, host: Hi folks, how are you doing?</s>Ms. WENDY RAQUEL ROBINSON (Actress): Great.</s>Mr. KYLE BOWSER (Co-producer, "The Bible Experien

In [ ]:
print(f"- The {dataset_id} dataset has {dataset.num_rows} examples.")
print(f"- Each example is a {type(dataset[0])} with a {type(dataset[0]['document'])} as value.")
print(f"- Examples look like this: {dataset[0]}")

- The ccdv/mediasum dataset has 443596 examples.
- Each example is a <class 'dict'> with a <class 'str'> as value.
- Examples look like this: {'document': 'FARAI CHIDEYA, host: Now, moving on, Forest Whitaker as Moses, Tisha Campbell Martin as Mary Magdalene - well, that\'s all in "The Bible Experience." A New Testament edition was released in 2006. This edition is billed as "The Complete Bible." It doesn\'t have one person reading the gospels. It features nearly 400 African-American artists, actors and ministers, plus sound effects.</s>FARAI CHIDEYA, host: Just listen to Blair Underwood\'s rendition of Jesus on the cross.</s>Mr. BLAIR UNDERWOOD (Actor): (As Jesus) My God, my God, why have you forsaken me?</s>FARAI CHIDEYA, host: Now, we\'ve got two people affiliated with the project with us today. Kyle Bowser, he co-produced "The Bible Experience" and actress Wendy Raquel Robinson, one of the actors in "The Bible Experience," and she also stars in the CW series, "The Game."</s>FARAI C

# 4.0 Helper Functions

In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.replace('\n', ' ')
    sentence = sentence.replace('?</s>', '?.')
    sentence = sentence.replace('</s>', '')
    sentence = sentence.replace("\'", "'")
    sentence = sentence.replace('--', '')
    sentence = sentence.replace('|', '')
    sentence = sentence.replace('/', '')
    sentence = sentence.replace('Dr.', 'Dr')
    sentence = sentence.replace('?.', '?. ')
    sentence = sentence.replace('.', '. ')
    sentence = sentence.replace('!', '!. ')
    return sentence

In [ ]:
def calculate_rouge(reference, predictions):
  rouge = evaluate.load('rouge')
  predictions = extracted_sentence_list
  references = orig_highlights_list
  results = rouge.compute(predictions=predictions,
                        references=references)
  print(results)

In [ ]:
"""
LexRank implementation
Source: https://github.com/crabcamp/lexrank/tree/dev
"""

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.special import softmax
import logging

logger = logging.getLogger(__name__)

def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (
        threshold is None
        or isinstance(threshold, float)
        and 0 <= threshold < 1
    ):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None',
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True, max_iter=10000):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    for _ in range(max_iter):
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)

    logger.warning("Maximum number of iterations for power method exceeded without convergence!")
    return eigenvector_next


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    # normalize probability distribution differently if we have negative transition values
    if np.min(weights_matrix) <= 0:
        return softmax(weights_matrix, axis=1)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

In [ ]:
# Trigram Blocking for Extractive Summarization

def trigram_blocking(input_text):

  trigram_master_list=[]
  trigram_temp_list=[]
  clean_list = []
  trigram_flag = []
  output_list = input_text

  #output_list = ['oh my god', 'oh my god' ,'lovely day today', 'This year is good', 'Oh my god' ]

  for idx, sentence in enumerate(output_list):
    if idx == 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)          # find all the trigram in text1
      #print(idx, sentence)
      for item in trigrams:
        #print(item)
        if item not in trigram_master_list:
          trigram_master_list.append(item)
      clean_list.append(sentence)
    elif idx > 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)
      #print(idx, sentence) 
      for item in trigrams:
        #print(item)
        trigram_temp_list.append(item)
      trigram_flag = []  
      for trigram_temp in trigram_temp_list:
        if trigram_temp in trigram_master_list:
          trigram_flag.append("Y")
        else:
          trigram_flag.append("N")
      if "N" in trigram_flag:
        clean_list.append(sentence)
        for i in trigram_temp_list:
          trigram_master_list.append(i)

  #print(trigram_master_list)
  #print(clean_list)

  return (clean_list)

#5.0 Create a Dataset for Model Building and Evaluation
 

In [ ]:
# Generate a dataset of 1000 examples for model prototyping

# DO NOT CHANGE THE CODE IN THIS CELL

size_of_dataset = 1000 # DO NOT CHANGE THIS VALUE
raw_dataset = dataset[0:size_of_dataset]
document_list =  raw_dataset['document']
summary_list =  raw_dataset['summary']

In [ ]:
# Pre-process text

def preprocess_text(sentence):
    sentence=str(sentence)
    sentence = sentence.replace('\n', ' ')
    sentence = sentence.replace('?</s>', '?.')
    sentence = sentence.replace('</s>', '')
    sentence = sentence.replace("\'", "'")
    sentence = sentence.replace('--', '')
    sentence = sentence.replace('|', '')
    sentence = sentence.replace('/', '')
    sentence = sentence.replace('Dr.', 'Dr')
    sentence = sentence.replace('?.', '?. ')
    sentence = sentence.replace('.', '. ')
    sentence = sentence.replace('!', '!. ')

    ### END YOUR CODE
    return sentence

In [ ]:
# Store in a List
clean_document_list = list(((map(preprocess_text, document_list))))
clean_summary_list = list(((map(preprocess_text, summary_list))))

In [ ]:
# Inspect Cleaned Text
clean_document_list[0]

'FARAI CHIDEYA, host: Now, moving on, Forest Whitaker as Moses, Tisha Campbell Martin as Mary Magdalene - well, that\'s all in "The Bible Experience. " A New Testament edition was released in 2006.  This edition is billed as "The Complete Bible. " It doesn\'t have one person reading the gospels.  It features nearly 400 African-American artists, actors and ministers, plus sound effects. FARAI CHIDEYA, host: Just listen to Blair Underwood\'s rendition of Jesus on the cross. Mr.  BLAIR UNDERWOOD (Actor): (As Jesus) My God, my God, why have you forsaken me?.  FARAI CHIDEYA, host: Now, we\'ve got two people affiliated with the project with us today.  Kyle Bowser, he co-produced "The Bible Experience" and actress Wendy Raquel Robinson, one of the actors in "The Bible Experience," and she also stars in the CW series, "The Game. "FARAI CHIDEYA, host: Hi folks, how are you doing?.  Ms.  WENDY RAQUEL ROBINSON (Actress): Great. Mr.  KYLE BOWSER (Co-producer, "The Bible Experience: The Complete Bi

In [ ]:
# Create a pandas dataframe to hold cleansed data
df = pd.DataFrame(list(zip(clean_document_list, clean_summary_list)),
               columns =['document', 'summary'])

In [ ]:
# write to a csv file
df.to_csv("cleaned_mediasum1000", index = False)

In [ ]:
# load the cleaned mediasum1000 into a Hugging Face Data Dictionary
dataset = load_dataset('csv', data_files = 'cleaned_mediasum1000', split='train' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fd7dc244812d1dcb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
# Inspect the data dictionary
dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 1000
})

#6.0 Create a Test Dataset for Model Evaluation

In [ ]:
# Generate a dataset of "x" examples for model evaluation

size_of_dataset = 1000 # change the value as you develop
small_dataset = dataset[0:size_of_dataset]


#7.0 Model Use Pre-Trained Sentence Transformer for Extractive Summarization #

    (NO FINE TUNING)

In [ ]:
# Install and import Sentence Transformer
!pip install -U sentence-transformers 
import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')
model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 42.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=498ab845b60139a6323fc0eb28cd1adeb73b5cee17c2771304021119e449a6b5
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

#8.0 Evaluate Rouge Across Eval Data Set

In [ ]:
# RougueL Scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge = evaluate.load('rouge')


In [ ]:
# Iterate through the dataset, extract the summary and compute RougeL, Cosine Similarity Scores

start = time.time()

# get article from dataset
input_article = small_dataset['document'][0:size_of_dataset]
#len(input_article)

# get summary from dataset
input_highlights = small_dataset['summary'][0:size_of_dataset]
#len(input_highlights)

# zip article and summary
zipped_input = zip(input_article, input_highlights)

# Empty List to Store Scores
rougeL_precision = []
rougeL_recall = []
rougeL_fmeasure = []

rouge_1_list = []
rouge_2_list = []
rouge_L_list = []

cosine_similarity_results = []

# Counter for Tracking Results
count = 1

# Extracted Highlights
output_highlights_list = []

for input_article, input_highlights in zipped_input:

  print('Example:', count)
  
# Tokenize the string texts  
  source_article_list = nltk.sent_tokenize(input_article)
  source_highlight_list = nltk.sent_tokenize(input_highlights)
  
# Join the list into a string
  formatted_source_article = " ".join(source_article_list)
  formatted_source_highlight = " ".join(source_highlight_list)
  
# get embeddings from model
  embeddings = model.encode(source_article_list, convert_to_tensor=True)

  output_list=[]

#Compute the pair-wise cosine similarities
  cos_scores = util.cos_sim(embeddings, embeddings).numpy()

#Compute the centrality for each sentence
  centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

#We argsort so that the first element is the sentence with the highest score
  most_central_sentence_indices = np.argsort(-centrality_scores)

  num_sentences = int(len(source_article_list) * 0.30) # 30% compression ration

  for idx in most_central_sentence_indices[0:num_sentences]:
    output_list.append(source_article_list[idx])

  result = output_list 

# note result is a string returned by model
  summary = "".join(result)

# Take the string from summary and convert to list of strings for each sentence
  extracted_sentence_list = nltk.sent_tokenize(summary)

# Apply Trigram Blocking
  result_list = trigram_blocking(extracted_sentence_list)
  summary = " ".join(result_list)
  output_highlights_list.append(summary)

  predictions = " ".join(result_list)
  references = formatted_source_highlight

  rougeL_scores = scorer.score(predictions,
                      references)
  
  pred = [predictions]
  ref = [references]

  rouge_results = rouge.compute(predictions=pred, references=ref)
  print(rouge_results)

  rouge_1_score = (rouge_results['rouge1'])
  rouge_2_score = (rouge_results['rouge2'])
  rouge_L_score = (rouge_results['rougeL'])

  rouge_1_list.append(rouge_1_score)
  rouge_2_list.append(rouge_2_score)
  rouge_L_list.append(rouge_L_score)

  precision = (rougeL_scores['rougeL'].precision)
  recall = (rougeL_scores['rougeL'].recall)
  fmeasure = (rougeL_scores['rougeL'].fmeasure)

  rougeL_precision.append(precision)
  rougeL_recall.append(recall)
  rougeL_fmeasure.append(fmeasure)

  # calculate cosine similarity
  doc1 = nlp((" ".join(predictions)))
  doc2 = nlp((" ".join(references)))
  cosine_similarity = doc1.similarity(doc2)
  cosine_similarity_results.append(cosine_similarity)

  count = count + 1

Example: 1
{'rouge1': 0.10372771474878445, 'rouge2': 0.048780487804878044, 'rougeL': 0.07455429497568881, 'rougeLsum': 0.07455429497568881}
Example: 2
{'rouge1': 0.041237113402061855, 'rouge2': 0.0103359173126615, 'rougeL': 0.028350515463917526, 'rougeLsum': 0.028350515463917526}
Example: 3
{'rouge1': 0.10625000000000001, 'rouge2': 0.04402515723270441, 'rougeL': 0.06875, 'rougeLsum': 0.06875}
Example: 4
{'rouge1': 0.13211845102505695, 'rouge2': 0.050343249427917625, 'rougeL': 0.08656036446469248, 'rougeLsum': 0.08656036446469248}
Example: 5
{'rouge1': 0.12578616352201258, 'rouge2': 0.02531645569620253, 'rougeL': 0.07547169811320754, 'rougeLsum': 0.07547169811320754}
Example: 6
{'rouge1': 0.07894736842105263, 'rouge2': 0.03736263736263736, 'rougeL': 0.054824561403508776, 'rougeLsum': 0.054824561403508776}
Example: 7
{'rouge1': 0.15165876777251186, 'rouge2': 0.009569377990430623, 'rougeL': 0.08530805687203792, 'rougeLsum': 0.08530805687203792}
Example: 8
{'rouge1': 0.11585365853658537, '

In [ ]:
# Print Compute Time
print('\nTime:', time.time() - start)


Time: 2907.3029823303223


In [ ]:
# Export extractive summary to a CSV

# get article from dataset
input_article_list = small_dataset['document'][0:size_of_dataset]

# get summary from dataset
input_highlights_list = small_dataset['summary'][0:size_of_dataset]

df = pd.DataFrame(list(zip(input_article_list, input_highlights_list, output_highlights_list)),
                  columns = ['orig_article', 'orig_summary', 'extracted_summary'])

# Edit this filepath to wherever you saved the data in your Drive
filepath = 'drive/My Drive/Colab_Notebooks_1/model_2a_extracted_mediasum1000.csv'

df.to_csv(filepath,index = False)

In [ ]:
# read back the csv file
data_import = pd.read_csv(filepath)        
#data_import.rename(columns = {'0':'orig_article', '1':'orig_summary', '2':'extracted_summary'}, inplace = True)

col1 = data_import.orig_article.values.tolist()
col2 = data_import.orig_summary.values.tolist()
col3 = data_import.extracted_summary.values.tolist()

In [ ]:
# Validate data_import
print(col1[0])
print("")

print(col2[0])
print("")

print(col3[0])


FARAI CHIDEYA, host: Now, moving on, Forest Whitaker as Moses, Tisha Campbell Martin as Mary Magdalene - well, that's all in "The Bible Experience. " A New Testament edition was released in 2006.  This edition is billed as "The Complete Bible. " It doesn't have one person reading the gospels.  It features nearly 400 African-American artists, actors and ministers, plus sound effects. FARAI CHIDEYA, host: Just listen to Blair Underwood's rendition of Jesus on the cross. Mr.  BLAIR UNDERWOOD (Actor): (As Jesus) My God, my God, why have you forsaken me?.  FARAI CHIDEYA, host: Now, we've got two people affiliated with the project with us today.  Kyle Bowser, he co-produced "The Bible Experience" and actress Wendy Raquel Robinson, one of the actors in "The Bible Experience," and she also stars in the CW series, "The Game. "FARAI CHIDEYA, host: Hi folks, how are you doing?.  Ms.  WENDY RAQUEL ROBINSON (Actress): Great. Mr.  KYLE BOWSER (Co-producer, "The Bible Experience: The Complete Bible")

In [ ]:
# Calculate Mean Rouge for Dataset

print("RougeL Precision Scores")
print(rougeL_precision)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_precision)))
print("")

print("RougeL Recall Scores")
print(rougeL_recall)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_recall)))
print("")

print("RougeL Fmeasure Scores")
print(rougeL_fmeasure)
print(len(rougeL_fmeasure))
print(np.mean(np.asarray(rougeL_fmeasure)))
print("")

print("Rouge 1 Scores")
print(rouge_1_list)
print(len(rouge_1_list))
print(np.mean(np.asarray(rouge_1_list)))
print("")

print("Rouge 2 Scores")
print(rouge_2_list)
print(len(rouge_2_list))
print(np.mean(np.asarray(rouge_2_list)))
print("")

print("Rouge L Scores")
print(rouge_L_list)
print(len(rouge_L_list))
print(np.mean(np.asarray(rouge_L_list)))
print("")


RougeL Precision Scores
[0.5348837209302325, 0.5, 0.4230769230769231, 0.4418604651162791, 0.4, 0.5952380952380952, 0.23684210526315788, 0.4473684210526316, 0.55, 0.4117647058823529, 0.44, 0.5, 0.575, 0.23255813953488372, 0.358974358974359, 0.2608695652173913, 0.4782608695652174, 0.43478260869565216, 0.5862068965517241, 0.3939393939393939, 0.27906976744186046, 0.30303030303030304, 0.34146341463414637, 0.3225806451612903, 0.21818181818181817, 0.7111111111111111, 0.42424242424242425, 0.42857142857142855, 0.3142857142857143, 0.6341463414634146, 0.3877551020408163, 0.2682926829268293, 0.3409090909090909, 0.2826086956521739, 0.7948717948717948, 0.5423728813559322, 0.37349397590361444, 0.5365853658536586, 0.2903225806451613, 0.5, 0.3225806451612903, 0.509090909090909, 0.6595744680851063, 0.64, 0.42857142857142855, 0.4666666666666667, 0.36363636363636365, 0.18518518518518517, 0.46511627906976744, 0.5, 0.45454545454545453, 0.627906976744186, 0.4117647058823529, 0.7307692307692307, 0.33333333333

In [ ]:
# Calculate Mean Cosine Similarity

print("Cosine Similarity")
print(np.mean(np.asarray(cosine_similarity_results)))
print("")



Cosine Similarity
0.9804626988613138



In [ ]:
dataset = load_dataset('csv', data_files = filepath, split='train' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-02b24c6db24ac611/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['orig_article', 'orig_summary', 'extracted_summary'],
    num_rows: 1000
})